# 容量制約つき施設配置問題

In [1]:
from gurobipy import *

In [2]:
def make_data():
    I,d = multidict({1:80, 2:270, 3:250, 4:160, 5:180})
    J,M,f = multidict({1:[500,1000], 2:[500,1000], 3:[500,1000]})
    c = {(1,1):4, (1,2):6, (1,3):9,
         (2,1):5, (2,2):4, (2,3):7,
         (3,1):6, (3,2):3, (3,3):4,
         (4,1):8, (4,2):5, (4,3):3,
         (5,1):10, (5,2):8, (5,3):4
        }
    return I,J,d,M,f,c

In [3]:
def flp(I,J,d,M,f,c):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y{}".format(j))
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x{}_{}".format(i,j))
    model.update()
    for i in I:
        model.addConstr(quicksum(x[i,j] for j in J) == d[i])
    for j in J:
        model.addConstr(quicksum(x[i,j] for i in I) <= M[j]*y[j])
    for (i,j) in x:
        model.addConstr(x[i,j] <= d[i]*y[j])
    model.setObjective(quicksum(f[j]*y[j] for j in J) +
                       quicksum(c[i,j]*x[i,j] for i in I for j in J), GRB.MINIMIZE)
    model.__data = x, y
    
    return model

In [4]:
I,J,d,c,f,M = make_data()
model = flp(I,J,d,c,f,M)
model.optimize()
for v in model.getVars():
    print(v.VarName, v.X)

Academic license - for non-commercial use only - expires 2021-06-06
Using license file /Users/yoshidayuto/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 23 rows, 18 columns and 63 nonzeros
Model fingerprint: 0x96676946
Variable types: 15 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [3e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 3e+02]
Presolve time: 0.00s
Presolved: 23 rows, 18 columns, 63 nonzeros
Variable types: 15 continuous, 3 integer (3 binary)

Root relaxation: objective 5.610000e+03, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5610.0000000 5610.00000  0.00%     -    0s

Explored 0 nodes (10 simplex iterat

# 強い定式化と弱い定式化

# k-メディアン問題

In [5]:
def kmedian(I,J,c,k):
    model = Model("k-median")
    x, y = {}, {}
    for j in J:
        y[j] = model.addConstr(vtype="B")
        for i in I:
            x[i,j] = model.addConstr(vtype="B")
    model.update()
    for i in I:
        model.addConstr(quicksum(x[i,j] for j in J) == 1)
        for j in J:
            model.addConstr(x[i,j] <= y[j])
    model.addConstr(quicksum(y[j] for j in J) == k)
    model.setObjective(quicksum(x[i,j]*c[i,j] for i in I for j in J), GRB.MINIMIZE)
    model.update()
    model.__data = x,y
    return model

# k-センター問題

In [6]:
def kcenter(I,J,c,k):
    model = Model("k-center")
    z = model.addVar(vtype="C")
    x, y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B")
        for i in I:
            x[i,j] = model.addVar(vtype="B")
    model.update()
    model.addConstr(quicksum(y[j] for j in J) == k)
    for i in I:
        model.addConstr(quicksum(x[i,j] for j in J) == 1)
        for j in J:
            model.addConstr(x[i,j] <= y[j])
            model.addConstr(c[i,j]*x[i,j] <= z)
    model.setObjective(z,GRB.MINIMIZE)
    model.update()
    model.__data = x,y
    return model

In [7]:
def k_cover(I,J,c,k):
    model = Model("k-center")
    z, y, x = {}, {}, {}
    for i in I:
        z[i] = model.addVar(vtype="B")
    for j in J:
        y = model.addVar(vtype="B")
        for i in I:
            x[i,j] = model.addVar(vtype="B")
    model.update()
    for i in I:
        model.addConstr(quicksum(x[i,j] for j in J) + z[i] == 1)
        for j in J:
            model.addConstr(x[i,j] <= y[j])
    model.addConstr(quicksum(y[j] for j in J) == k)
    model.setObjective(quicksum(z[i] for i in I), GRB.MINIMIZE)
    model.update()
    model.__data = x,y,z
    return model

In [8]:
def solve_kcenter(I,J,c,k):
    model = k_center(I,J,c,k)
    model.Params.Cntoff = .1
    x,y,z = model.__data
    LB = 0
    UB = max(c[i,j] for (i,j) in c)
    while UB - LB > 1.e-4:
        theta = (UB+LB) / 2.
        # 頂点被覆問題の求解
        for j in J:
            for i in I:
                if c[i,j] > theta:
                    x[i,j].UB = 0
                else:
                    x[i,j].UB = 1.0
        model.update()
        model.optimize()
        infeasibility = sum([z[i].X for i in I])
        if infeasibility > 0:
            LB = theta
        else:
            UB = theta